<a href="https://colab.research.google.com/github/pcmbs/preset-embedding_audio-model-colab/blob/main/TAL_NoiseMaker_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Download TAL-NoiseMaker and presets

In [1]:
!wget -c https://tal-software.com/downloads/plugins/TAL-NoiseMaker_64_linux.zip && unzip -o TAL-NoiseMaker_64_linux.zip && rm TAL-NoiseMaker_64_linux.zip
!mkdir presets && cd presets && wget 'https://tal-software.com//downloads/presets/TAL-NoiseMaker%20vst3.zip' && unzip -q -o "TAL-NoiseMaker vst3.zip"

--2023-07-13 09:24:49--  https://tal-software.com/downloads/plugins/TAL-NoiseMaker_64_linux.zip
Resolving tal-software.com (tal-software.com)... 172.67.167.238, 104.21.13.116, 2606:4700:3037::6815:d74, ...
Connecting to tal-software.com (tal-software.com)|172.67.167.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7048068 (6.7M) [application/x-zip-compressed]
Saving to: ‘TAL-NoiseMaker_64_linux.zip’

TAL-NoiseMaker_64_l 100%[===================>]   6.72M  4.77MB/s    in 1.4s    

2023-07-13 09:24:52 (4.77 MB/s) - ‘TAL-NoiseMaker_64_linux.zip’ saved [7048068/7048068]

Archive:  TAL-NoiseMaker_64_linux.zip
  inflating: TAL-NoiseMaker/ReadmeLinux.txt  
  inflating: TAL-NoiseMaker/TAL-NoiseMaker.clap  
   creating: TAL-NoiseMaker/TAL-NoiseMaker.vst3/
   creating: TAL-NoiseMaker/TAL-NoiseMaker.vst3/Contents/
   creating: TAL-NoiseMaker/TAL-NoiseMaker.vst3/Contents/x86_64-linux/
  inflating: TAL-NoiseMaker/TAL-NoiseMaker.vst3/Contents/x86_64-linux/TAL-NoiseMaker.

Install dawdreamer

In [2]:
!pip install dawdreamer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 MB 19.4 MB/s eta 0:00:00


Setup dawdreamer

In [3]:
import dawdreamer as daw
import numpy as np

SAMPLE_RATE = 44100
BUFFER_SIZE = 128  # Parameters will undergo automation at this buffer/block size.
PPQN = 960  # Pulses per quarter note.

SYNTH_PLUGIN = "TAL-NoiseMaker/TAL-NoiseMaker.vst3"

engine = daw.RenderEngine(SAMPLE_RATE, BUFFER_SIZE)
# Make a processor and give it the unique name "my_synth", which we use later.
synth = engine.make_plugin_processor("tal_noisemaker", SYNTH_PLUGIN)
assert synth.get_name() == "tal_noisemaker"

synth name: tal_noisemaker
synth num inputs: 2
synth num outputs: 2


# Synth info

In [14]:
print(f"synth name: {synth.get_name()}")
print(f"synth num inputs: {synth.get_num_input_channels()}")
print(f"synth num outputs: {synth.get_num_output_channels()}")

synth name: tal_noisemaker
synth num inputs: 2
synth num outputs: 2


In [13]:
params_list = synth.get_parameters_description()
for param_dict in params_list[1:86]: # params outside this range are not relevant
    print(param_dict)

{'index': 1, 'name': 'Master Volume', 'numSteps': 2147483647, 'isBoolean': False, 'isDiscrete': False, 'label': '', 'category': 'genericParameter', 'text': '0.4080000', 'isMetaParameter': False, 'isAutomatable': True, 'defaultValue': 0.4999000132083893, 'defaultValueText': '0.4999000', 'min': '0.0000000', 'max': '1.0000000', 'valueStrings': []}
{'index': 2, 'name': 'Filter Type', 'numSteps': 2147483647, 'isBoolean': False, 'isDiscrete': False, 'label': '', 'category': 'genericParameter', 'text': '0.0000000', 'isMetaParameter': False, 'isAutomatable': True, 'defaultValue': 1.0, 'defaultValueText': '1.0000000', 'min': '0.0000000', 'max': '1.0000000', 'valueStrings': []}
{'index': 3, 'name': 'Filter Cutoff', 'numSteps': 2147483647, 'isBoolean': False, 'isDiscrete': False, 'label': '', 'category': 'genericParameter', 'text': '1.0000000', 'isMetaParameter': False, 'isAutomatable': True, 'defaultValue': 1.0, 'defaultValueText': '1.0000000', 'min': '0.0000000', 'max': '1.0000000', 'valueStrin

# Functions definition

In [17]:
def generate_sound_from_preset(preset_path: str) -> np.ndarray:
    synth.load_state(preset_path)
    return _render_preset()

def generate_random_sound() -> np.ndarray:
    for i in range(1, 86):
        synth.set_parameter(i, np.random.uniform(0, 1))

    return _render_preset()

def _render_preset() -> np.ndarray:
    synth.add_midi_note(60, 100, 0.0, 3.0)  # (MIDI note, velocity, start, duration)
    # don't do reverb
    graph = [
        (synth, []),  # synth takes no inputs, so we give an empty list.
    ]
    engine.load_graph(graph)
    engine.render(4)
    output = engine.get_audio()

    return output

def print_audio_stats(audio: np.ndarray) -> None:
    print("mean", np.mean(np.abs(audio)))
    print("max", np.max(audio))
    print("min", np.min(audio))
    print("std", np.std(audio))


# Tests

check if vst3 presets are correctly loaded and rendered

In [19]:
import glob
import IPython.display as ipd
import random

# Shuffle the order of the presets
random.seed(0)
presets = list(glob.glob("presets/*.vstpreset"))
random.shuffle(presets)


preset = random.choice(presets)
print(preset)
output = generate_sound_from_preset(preset)
print_audio_stats(output)

preset = random.choice(presets)
print(preset)
generate_sound_from_preset(preset)
print_audio_stats(output)

presets/LD Technoshocker FN.vstpreset
mean 0.1021148
max 0.2887225
min -0.25095552
std 0.13524932
presets/FX C64 FN.vstpreset
mean 0.1021148
max 0.2887225
min -0.25095552
std 0.13524932


check if setting up each params individually change the synth state

In [21]:
# first random preset
output = generate_random_sound()
print_audio_stats(output)
ipd.Audio(np.mean(output, axis=0), rate=int(SAMPLE_RATE))

mean 0.0017170857
max 0.0061462144
min -0.0062858416
std 0.0020645224


In [22]:
# second random preset
output = generate_random_sound()
print_audio_stats(output)
ipd.Audio(np.mean(output, axis=0), rate=int(SAMPLE_RATE))

mean 0.011012713
max 0.056854054
min -0.063820295
std 0.012853395
